In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
import copy
from tqdm import tqdm_notebook

import Config
import Dataloader as DL
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC


In [3]:
dl = DL.Dataloader()
nFeatures, nClasses, traindata, trainlabels, testdata, testlabels = dl.getParam()
traindata = traindata[:20000]

Loading dataset MNIST from MNIST
Loading train data... train data of shape (60000, 784) loaded
Loading test data...  test  data of shape (10000, 784) loaded
Data Loaded. Num of features = 784 Num of Classes = 10

In [4]:
param = Config.config
param["nFeatures"] = nFeatures
param["nClasses"] = nClasses
print(param)

{'data_location': '../dataset/', 'directory': 'MNIST', 'dataset': 'MNIST', 'D': 10000, 'vector': 'Gaussian', 'mu': 0, 'sigma': 1, 'binarize': 0, 'lr': 0.037, 'sparse': 0, 's': 0.1, 'binaryModel': 0, 'width': None, 'height': None, 'nLayers': 5, 'uniform_dim': 1, 'uniform_ker': 1, 'dArr': None, 'k': 5, 'kArr': None, 'one_shot': 0, 'data_percentages': [1.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5], 'train_percent': 1, 'dropout': 0, 'drop_percentages': [0, 0.1, 0.2, 0.5], 'dropout_rate': 0, 'update_type': <Update_T.FULL: 1>, 'iter_per_trial': 3, 'iter_per_encoding': 5, 'epochs': 50, 'nFeatures': 784, 'nClasses': 10}


In [5]:
################# VANILLA #################
hdb = HDB.HD_basis(HDB.Generator.Vanilla, param)
basis = hdb.getBasis()
bid = hdb.getParam()["id"]
# Update param with bid
param = hdb.getParam()
print(bid)

Generating vanilla HD basis of shape... 

(10000, 784)
Encoding time: 0.36095499992370605 
Dumping basis into file: base_8.pkl 


8


In [6]:
# Retrieve info upto basis generator, given correct bid
#bid = 6679
#basis, param = HDB.loadBasis("base_%d.pkl"%bid)

In [7]:
hde = HDE.HD_encoder(basis)

trainencoded = hde.encodeData(traindata)
#HDE.saveEncoded(trainencoded, trainlabels, bid, "train")

testencoded = hde.encodeData(testdata)
#HDE.saveEncoded(testencoded, testlabels, bid, "test")

Encoding data of shape (20000, 784)


Time spent: 1113 sec
Dumping data into encoded_8_train.pkl 
Encoding data of shape (10000, 784)


Time spent: 584 sec
Dumping data into encoded_8_test.pkl 


'encoded_8_test.pkl'

In [8]:
# Retrieve info upto encoder, given correct bid
#bid = 6679
#basis, param = HDB.loadBasis("base_%d.pkl"%bid)
#trainencoded, trainlabels = HDE.loadEncoded("encoded_%d_train.pkl"%bid)
#testencoded, testlabels = HDE.loadEncoded("encoded_%d_test.pkl"%bid)

[autoreload of HD_classifier failed: Traceback (most recent call last):
  File "c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\IPython\extensions\autoreload.py", line 434, in superreload
    module = reload(module)
  File "c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line

In [9]:
# train data 
def train(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config):
    train_acc = []
    test_acc = []
    for _ in tqdm_notebook(range(param["epochs"]), desc='epochs'):
        train_acc.append(hdc.fit(traindata, trainlabels, param))
        test_acc.append(hdc.test(testdata, testlabels))
        if len(train_acc) % 5 == 0:
            print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
        if train_acc[-1] == 1:
            print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
            break
    return np.asarray(train_acc), np.asarray(test_acc)

In [10]:
def dump_log(param, train_acc, test_acc, filename):
    joblib.dump((param, train_acc, test_acc), open(filename, "wb"), compress=True)

In [11]:
# Should have 90%
hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = train(hdc, trainencoded, trainlabels, testencoded, testlabels)
#dump_log(param, train_acc, test_acc, "Baseline_"+param["id"]+".pickle")

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037)] 


Train: 0.992100 	 	 Test: 0.953600
Train: 0.996050 	 	 Test: 0.961700
Train: 0.998450 	 	 Test: 0.967200
Train: 0.999750 	 	 Test: 0.968500
Train: 1.000000 	 	 Test: 0.968500


In [41]:
rank_test_acc = dict()
for k in [1, 2, 5, 10, 20, 40]:
    rank_test_acc[k] = hdc.test_rank(testencoded, testlabels, k)

In [42]:
print(rank_test_acc)
print(test_acc)

{1: 0.9685, 2: 0.9605, 5: 0.9622, 10: 0.9622, 20: 0.9617, 40: 0.9629}
[0.9041 0.9417 0.9637 0.9538 0.9536 0.9583 0.9666 0.9547 0.9607 0.9617
 0.9639 0.9591 0.9655 0.9656 0.9672 0.9654 0.9686 0.9641 0.9607 0.9685
 0.9685]


In [50]:
# k shit???
param_k = copy.deepcopy(param)
param_k["D"] = param["D"]//k

hdb_k = []
hde_k = []
hdc_k = []
rank_class_k = np.zeros((len(testencoded), param_k["nClasses"]))

# Create k models
for i in range(k):
    hdb_k.append(HDB.HD_basis(HDB.Generator.Vanilla, param_k))
    basis = hdb_k[-1].getBasis()
    param_k = hdb_k[-1].getParam()
    hde_k.append(HDE.HD_encoder(basis))
    trainencoded_k = hde_k[-1].encodeData(traindata)
    testencoded_k = hde_k[-1].encodeData(testdata)
    hdc_k.append(HDC.HD_classifier(param_k["D"], param_k["nClasses"], 0))    # 0 for bid because don't care 
    train(hdc_k[-1], trainencoded_k, trainlabels, testencoded_k, testlabels)
    
    rankings = []
    # generate arrays of rankings for test data
    for i in range(len(testencoded_k)):  
        rankings.append(HDC.rank_class(hdc_k[-1].classes, testencoded_k[i]))
    # Accumulate rankings
    rank_class_k = rank_class_k + rankings

Generating vanilla HD basis of shape... 

(250, 784)
Encoding time: 0.03206801414489746 
Dumping basis into file: base_9264.pkl 
Encoding data of shape (20000, 784)


Time spent: 25 sec
Encoding data of shape (10000, 784)


Time spent: 12 sec
c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037)] 


...

(250, 784)
Encoding time: 0.03240489959716797 
Dumping basis into file: base_1862.pkl 
Encoding data of shape (20000, 784)


Time spent: 28 sec
Encoding data of shape (10000, 784)


Time spent: 13 sec


Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037)] 


Train: 0.839600 	 	 Test: 0.826100
Train: 0.842650 	 	 Test: 0.853500
Train: 0.845250 	 	 Test: 0.827600
Train: 0.844950 	 	 Test: 0.832700
Train: 0.844100 	 	 Test: 0.844500
Train: 0.845250 	 	 Test: 0.840200
Train: 0.845300 	 	 Test: 0.828500
Train: 0.847650 	 	 Test: 0.843300
Train: 0.844750 	 	 Test: 0.849300
Train: 0.843800 	 	 Test: 0.840800



In [51]:
print(rank_class_k)

[[273. 365. 220. ...  41. 232. 132.]
 [216. 283.  68. ... 339. 174. 310.]
 [374.  41. 164. ... 195. 184. 281.]
 ...
 [356. 374. 272. ... 176. 132.  96.]
 [291. 292. 270. ... 286. 118. 249.]
 [205. 362. 126. ... 325. 226. 254.]]


In [52]:
guesses = [np.argmin(rank_class) for rank_class in rank_class_k]
correct = 0
for i in range(len(guesses)):
    if guesses[i] == testlabels[i]:
        correct += 1
print(correct/(len(guesses)))

0.9535
